In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import tensorflow as tf

from src.loaders.data_loader import load_train_data, load_basic_preprocessed_train, load_test_data
from src.preprocessors.preprocessors import split_train_valid_test, do_common_preprocess, apply_minmax_scale

from src.model.multiple_output.convolution import Convolution2DVarious
from src.loaders.data_loader import load_test_features, load_basic_preprocessed_predict, load_submission_data
from src.loaders.window_generator import WindowGenerator
from src.trainers import compile_and_fit_with_pinball_loss

df = load_train_data()
df = do_common_preprocess(df)

np.random.seed(42)
tf.random.set_seed(42)

In [2]:
cutter = [
    'DHI', 
    'DNI', 
#     'WS', 
#     'RH', 
#     'T', 
#     'Hour_sin',
#     'Hour_cos', 
    'GHI', 
#     'TARGET_ROLLING_MEAN_3_shift_1',
#     'TARGET_ROLLING_MEAN_5_shift_2', 
#     'TARGET_ROLLING_MEAN_11_shift_5',
#     'TARGET_ROLLING_MEAN_23_shift_11', 
#     'TARGET_ROLLING_MEAN_47_shift_23',
    'scaled_TARGET',
    "TARGET",
]



scaler = "minmax"
days = 7

In [3]:
def preprocess(train_df, valid_df, test_df, cutter):
    train_target = train_df["TARGET"]
    valid_target = valid_df["TARGET"]
    test_target = test_df["TARGET"]
    
    train_df, valid_df, test_df = apply_minmax_scale(train, valid, test)
    
    train_df["scaled_TARGET"] = train_df["TARGET"]
    valid_df["scaled_TARGET"] = valid_df["TARGET"]
    test_df["scaled_TARGET"] = test_df["TARGET"]
    
    train_df["TARGET"] = train_target
    valid_df["TARGET"] = valid_target
    test_df["TARGET"] = test_target
    
    train_df = train_df[cutter]
    valid_df = valid_df[cutter]
    test_df = test_df[cutter]
    
    return train_df, valid_df, test_df

In [4]:
def learn(days, train, valid, test):
    print('learn', train.shape, valid.shape, test.shape)

    one_days_window_label_columns = WindowGenerator(
        train,
        valid,
        test,
        input_width=48 * days,
        label_width=96,
        shift=96,
        sequence_stride=1,
        label_columns=["TARGET"]
        
    )
    submission_df = load_submission_data()
    predict = load_test_data()
    predict = do_common_preprocess(predict)
    _, predict, _ = apply_minmax_scale(train, predict, predict)
    
    predict["scaled_TARGET"] = predict["TARGET"]
    predict = predict[cutter]
    predict.drop("TARGET", axis=1, inplace=True)
    predict = load_test_features(predict, 48 * days)
    
    total_error = 0
    
    for i in range(1, 10):
        q = i/10
        print(q)
        
        conv_various = Convolution2DVarious(days, 96, train.shape[-1] - 1)
    
        compile_and_fit_with_pinball_loss(conv_various, one_days_window_label_columns, q)
        total_error += conv_various.evaluate(one_days_window_label_columns.test, verbose=0)[0]
        predict_np = predict.reshape(-1, days, 48, train.shape[-1] - 1)
        pred_y = conv_various.predict(predict_np)[:, :, -1]
        submission_df[f"q_{q}"] = pred_y.reshape(-1)

    return total_error, submission_df

In [5]:
df.shape[0]

52560

In [ ]:
length = df.shape[0]
boundary = [
    (slice(0, int(length * 0.6)),slice(int(length * 0.6), int(length * 0.9)), slice(int(length * 0.9), int(length * 1.0))),
    (slice(int(length * 0.1), int(length * 0.7)),slice(int(length * 0.7), int(length * 1)), slice(int(length * 0), int(length * 0.1))),    
    (slice(int(length * 0.3), int(length * 0.9)),slice(int(length * 0.9), int(length * 1)), slice(int(length * 0), int(length * 0.3))),
    (slice(int(length * 0.4), int(length * 1)),slice(int(length * 0), int(length * 0.3)), slice(int(length * 0.3), int(length * 0.4))),
]

total_error = 0
result_list = []

for train_slice, valid_slice, test_slice in boundary:
    train, valid, test = df[train_slice], df[valid_slice], df[test_slice]
    train, valid, test = preprocess(train, valid, test, cutter)
    
    
    print(train.shape, valid.shape, test.shape)
    error, result_df = learn(days, train, valid, test)
    total_error += error
    result_list.append(result_df)

(31536, 5) (15768, 5) (5256, 5)
learn (31536, 5) (15768, 5) (5256, 5)
0.1
Epoch 1/1000
973/973 [==============================] - 31s 21ms/step - loss: 1.7470 - mean_absolute_error: 16.6764 - mean_squared_error: 854.9351 - _pinball_loss: 1.7470 - val_loss: 1.5392 - val_mean_absolute_error: 13.8229 - val_mean_squared_error: 607.7477 - val__pinball_loss: 1.5389
Epoch 2/1000
973/973 [==============================] - 9s 9ms/step - loss: 1.5712 - mean_absolute_error: 13.9441 - mean_squared_error: 606.5827 - _pinball_loss: 1.5712 - val_loss: 1.5047 - val_mean_absolute_error: 13.2878 - val_mean_squared_error: 562.6301 - val__pinball_loss: 1.5046
Epoch 3/1000
973/973 [==============================] - 9s 9ms/step - loss: 1.5412 - mean_absolute_error: 13.4836 - mean_squared_error: 568.5994 - _pinball_loss: 1.5412 - val_loss: 1.4901 - val_mean_absolute_error: 13.0711 - val_mean_squared_error: 544.6246 - val__pinball_loss: 1.4900
Epoch 4/1000
973/973 [==============================] - 9s 9ms/ste

973/973 [==============================] - 9s 9ms/step - loss: 1.4490 - mean_absolute_error: 12.2643 - mean_squared_error: 490.8236 - _pinball_loss: 1.4490 - val_loss: 1.4315 - val_mean_absolute_error: 12.2946 - val_mean_squared_error: 498.6487 - val__pinball_loss: 1.4310
Epoch 30/1000
973/973 [==============================] - 9s 9ms/step - loss: 1.4473 - mean_absolute_error: 12.2410 - mean_squared_error: 489.3484 - _pinball_loss: 1.4473 - val_loss: 1.4312 - val_mean_absolute_error: 12.2805 - val_mean_squared_error: 497.5661 - val__pinball_loss: 1.4317
Epoch 31/1000
973/973 [==============================] - 9s 9ms/step - loss: 1.4462 - mean_absolute_error: 12.2234 - mean_squared_error: 487.9773 - _pinball_loss: 1.4462 - val_loss: 1.4286 - val_mean_absolute_error: 12.0912 - val_mean_squared_error: 487.3435 - val__pinball_loss: 1.4289
Epoch 32/1000
973/973 [==============================] - 9s 9ms/step - loss: 1.4467 - mean_absolute_error: 12.2100 - mean_squared_error: 487.3475 - _pinb

0.3
Epoch 1/1000
973/973 [==============================] - 10s 9ms/step - loss: 4.0869 - mean_absolute_error: 11.9293 - mean_squared_error: 478.2967 - _pinball_loss: 4.0869 - val_loss: 2.9247 - val_mean_absolute_error: 7.0837 - val_mean_squared_error: 169.4677 - val__pinball_loss: 2.9249
Epoch 2/1000
973/973 [==============================] - 9s 9ms/step - loss: 3.0204 - mean_absolute_error: 7.5121 - mean_squared_error: 182.1312 - _pinball_loss: 3.0205 - val_loss: 2.8539 - val_mean_absolute_error: 6.9870 - val_mean_squared_error: 169.7069 - val__pinball_loss: 2.8531
Epoch 3/1000
973/973 [==============================] - 9s 9ms/step - loss: 2.9630 - mean_absolute_error: 7.3434 - mean_squared_error: 178.6871 - _pinball_loss: 2.9630 - val_loss: 2.8104 - val_mean_absolute_error: 6.9684 - val_mean_squared_error: 169.8230 - val__pinball_loss: 2.8111
Epoch 4/1000
973/973 [==============================] - 9s 9ms/step - loss: 2.9254 - mean_absolute_error: 7.2478 - mean_squared_error: 176.381

Epoch 14/1000
973/973 [==============================] - 10s 11ms/step - loss: 2.9314 - mean_absolute_error: 5.9090 - mean_squared_error: 137.2017 - _pinball_loss: 2.9314 - val_loss: 2.8906 - val_mean_absolute_error: 5.9104 - val_mean_squared_error: 139.6640 - val__pinball_loss: 2.8918
0.5
Epoch 1/1000
973/973 [==============================] - 11s 11ms/step - loss: 5.1780 - mean_absolute_error: 10.3560 - mean_squared_error: 360.0397 - _pinball_loss: 5.1780 - val_loss: 3.0489 - val_mean_absolute_error: 6.0978 - val_mean_squared_error: 154.7707 - val__pinball_loss: 3.0485
Epoch 2/1000
973/973 [==============================] - 11s 11ms/step - loss: 3.1377 - mean_absolute_error: 6.2754 - mean_squared_error: 157.9557 - _pinball_loss: 3.1377 - val_loss: 2.9278 - val_mean_absolute_error: 5.8557 - val_mean_squared_error: 149.1639 - val__pinball_loss: 2.9283
Epoch 3/1000
973/973 [==============================] - 10s 10ms/step - loss: 3.0476 - mean_absolute_error: 6.0951 - mean_squared_error:

Epoch 16/1000
973/973 [==============================] - 10s 10ms/step - loss: 2.5531 - mean_absolute_error: 5.8401 - mean_squared_error: 170.0758 - _pinball_loss: 2.5531 - val_loss: 2.5000 - val_mean_absolute_error: 5.7738 - val_mean_squared_error: 170.7859 - val__pinball_loss: 2.5004
0.7
Epoch 1/1000
973/973 [==============================] - 11s 11ms/step - loss: 5.1146 - mean_absolute_error: 11.6212 - mean_squared_error: 363.9491 - _pinball_loss: 5.1146 - val_loss: 2.4385 - val_mean_absolute_error: 6.9715 - val_mean_squared_error: 202.0151 - val__pinball_loss: 2.4384
Epoch 2/1000
973/973 [==============================] - 10s 11ms/step - loss: 2.4937 - mean_absolute_error: 7.0758 - mean_squared_error: 211.3065 - _pinball_loss: 2.4937 - val_loss: 2.3270 - val_mean_absolute_error: 6.8882 - val_mean_squared_error: 210.8547 - val__pinball_loss: 2.3283
Epoch 3/1000
973/973 [==============================] - 10s 11ms/step - loss: 2.3765 - mean_absolute_error: 6.8058 - mean_squared_error:

...

973/973 [==============================] - 18s 19ms/step - loss: 2.7534 - mean_absolute_error: 5.5069 - mean_squared_error: 146.3172 - _pinball_loss: 2.7534 - val_loss: 3.1294 - val_mean_absolute_error: 6.2588 - val_mean_squared_error: 169.9986 - val__pinball_loss: 3.1288loss: 2.8586 - mean_absolute_error: 5.7171 - mean_squared_error:  - ETA: 11s - loss - ETA - ETA: 5s - loss: 2.7558 - mean_absolute_error: 5.5116 - mean_squared_error: 146.3769 - _pinball_loss - ETA: 2s - loss: 2.7540 - mean_absolute_error: 5.5081  - ETA: 0s - loss: 2.7537 - mean_absolute_error: 5.5075 - mean_squared_error: 146.32 - ETA: 0s - loss: 2.7535 - mean_absolute_error: 5.5071 - mean_squared_error: 146.3215 - _pinball_
Epoch 10/1000
973/973 [==============================] - 20s 20ms/step - loss: 2.7357 - mean_absolute_error: 5.4714 - mean_squared_error: 145.3768 - _pinball_loss: 2.7357 - val_loss: 3.1259 - val_mean_absolute_error: 6.2517 - val_mean_squared_error: 169.2834 - val__pinball_loss: 3.1263.4714 - mean

973/973 [==============================] - 22s 21ms/step - loss: 5.2484 - mean_absolute_error: 10.4463 - mean_squared_error: 348.2077 - _pinball_loss: 5.2484 - val_loss: 3.1379 - val_mean_absolute_error: 7.0543 - val_mean_squared_error: 199.3800 - val__pinball_loss: 3.13785s - loss: 10.8808 - mean_absolute_error: 18.3438 - mean_squared_error: 989.6454 - _pinball_loss: 10.880 - ETA: 15s - loss: 10.8210 - mean_absolute_error: 18.2717 - mean_ - ETA: 13s - loss: 9.2393 - mean_absolute_error: 16.3749 - mean_squared_error: 749.6640 - _pinball_lo - ETA: 13s - loss: 8.9882 - mean_absolute_error: 16.0543 - mean - ETA: 13s - loss: 8.2711 - mean_abso - ETA: 11s - loss: 7.3448 - mean_absolute_error: 13.7332 - mean_squared_error: 533.4239 - _pinball_loss - ETA: 11s - loss: 7.2645 - mean_absolute_e - ETA: 9s - loss: 6.5723 - mean_absolute_error: 12.5 - ETA: 5s - loss: 5.9369 - mean_absolute_error: 11.5571 - mean_squared_error: 404.4793 - _pinball_loss: 5.93 - ETA: 5s - loss: 5.9198 - mean_absolute_e

973/973 [==============================] - 19s 20ms/step - loss: 2.4602 - mean_absolute_error: 5.6445 - mean_squared_error: 170.4052 - _pinball_loss: 2.4602 - val_loss: 2.8179 - val_mean_absolute_error: 6.5256 - val_mean_squared_error: 197.3381 - val__pinball_loss: 2.8175- mean_absolute_error: 5.6617 - mean_squared_error: 171.4393 - _pinball_loss:  - ETA: 6s - loss: 2.4672 - mean_absolute_error: 5.6608 - mean_squared_error - ETA: 5s - loss: 2.4657 - mean_absolute_error: 5.6575 - mean_s - - ETA: 1s - loss: 2.4612 - mean_absolute_error: 5.6468 - mean_squared_error: 170.5178 - _pinball_loss:  - ETA: 1s - loss: 2.4611 - mean_absolute_error: 5.6466 - mean_squared_error: 17 - ETA: 0s - loss: 2.4605 - mean_absolute_error: 5.6453 - mean_squared_error: 170.4389 - _pinb - ETA: 0s - loss: 2.4603 - mean_absolute_error: 5.6447 - mean_squared_error: 170.4136 - _pinball_loss: 
Epoch 14/1000
973/973 [==============================] - 19s 19ms/step - loss: 2.4490 - mean_absolute_error: 5.6217 - mean_sq

973/973 [==============================] - 20s 20ms/step - loss: 2.1099 - mean_absolute_error: 6.1714 - mean_squared_error: 197.7520 - _pinball_loss: 2.1099 - val_loss: 2.4407 - val_mean_absolute_error: 7.1817 - val_mean_squared_error: 236.8203 - val__pinball_loss: 2.44081225 - mean_absolute_error: 6.1875 - mean_squared_error: 196.3390 - _pinball_loss: 2.1 - ETA: 12s - loss: 2.1217 - mean_absolute_error: 6.1859 - mean_squared_error: 196.3583 - _pinbal - ETA: 11s - loss: 2.1189 - mean_absolute_error: 6.1809 - mean_squared_error: 196.4904  - ETA: 10s - loss: 2.1171 - mean_absolute_error: 6.1799  - ETA
Epoch 7/1000
973/973 [==============================] - 19s 20ms/step - loss: 2.0829 - mean_absolute_error: 6.1168 - mean_squared_error: 197.7254 - _pinball_loss: 2.0829 - val_loss: 2.4600 - val_mean_absolute_error: 7.1300 - val_mean_squared_error: 230.6990 - val__pinball_loss: 2.4604934 - mean_absolute_error: 6.1411 - mean_squared_error: 197.3515 - _pinball_ - ETA: 13s - loss: 2. - ETA: 6s

973/973 [==============================] - 20s 21ms/step - loss: 1.9774 - mean_absolute_error: 5.8953 - mean_squared_error: 193.8348 - _pinball_loss: 1.9774 - val_loss: 2.3569 - val_mean_absolute_error: 7.0384 - val_mean_squared_error: 233.1306 - val__pinball_loss: 2.3582loss: 2.0629 - mean_absolute_error: 6.1713 - mean_squared_error: 192.8685 - _pinball_loss: 2.062 - ETA: 22s - loss: 2.0633 - mean_absolute_error: 6.1676 - mean_squared_error: 193.3812 - _pinball_loss: 2 - ETA: 19s - loss: 2.0741 - mean_absolute_error: 6.1939 - mean_ - ETA: 16s - loss: 2.0044 - mean_absolute_error: 5.9735 - mean_squared_error: 194.1232 - _pinball_loss: 2.0 - ETA: 16s - loss: 2.0029 - mean_absolute_error: 5.9690 - mean_squared_error: 194.0728 -  - ETA: 15s - loss: 1.9940 - mean_absolute_error: 5.9431 - mean_squared_error: 193.7041 - _pinball_los - ETA: 14s - loss: 1.9910 - mean_absolute_error: 5.9345 - mean_squared_error: 193.6309 - _pinball_loss: - ETA: 14s - loss: 1.9897 - mean_absolute_error: 5.9307 -

In [ ]:
total_error/ 4

In [ ]:
submission_df = load_submission_data()

for result_df in result_list:
    for i in range(1, 10):
        q = i/10
        submission_df[f"q_{q}"] += result_df[f"q_{q}"]
                      
for i in range(1, 10):
    q = i/10
    submission_df[f"q_{q}"] = submission_df[f"q_{q}"] / 4